In [3]:
%matplotlib inline

from __future__ import print_function
from sklearn import preprocessing
from time import time
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.kernel_ridge import KernelRidge
import numpy as np
import sklearn.decomposition
import sklearn.metrics
from sklearn import gaussian_process
from sklearn import cross_validation
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [4]:
np.random.seed(1234)

In [5]:
#load 2015 and 2016 data into train and test data set 
horseData2015 = np.array(pd.read_excel('HorseCTrain.xlsx').values)
horseData2015 = np.vstack({tuple(row) for row in horseData2015})

horseData2016 = np.array(pd.read_excel('HorseCTest.xlsx').values)
horseData2016 = np.vstack({tuple(row) for row in horseData2016})

x_train = horseData2015[:, :-1];
y_train = horseData2015[:, -1:horseData2015[0].size];

x_test = horseData2016[:, :-1];
y_test = horseData2016[:, -1:horseData2016[0].size];

#smoooth y for both training and testing data
y_train = y_train.reshape(len(y_train),).astype(int) + (np.random.random_sample((len(y_train),))-1) + np.random.random_sample((len(y_train),))
y_test = y_test.reshape(len(y_test),).astype(int) + (np.random.random_sample((len(y_test),))-1) + np.random.random_sample((len(y_test),))

#smooth x_train for grid_search
X_train_smooth = preprocessing.scale(x_train)

In [6]:
param_grid = {"theta0": [0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04, 0.045, 0.05, 0.055, 0.06, 0.065, 0.07],
              "nugget": [0.005, 0.01, 0.015, 0.02, 0.025, 0.03]}

reg_types = ['constant', 'linear']
corr_types = ['linear', 'absolute_exponential', 'cubic', 'squared_exponential']

num_top = 5
top_params = np.zeros((num_top,2))

In [7]:
def report(results, top_params):
    for i in range(1, num_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                 results['mean_test_score'][candidate],
                 results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
            top_params[i-1][0] = results["params"][candidate]['nugget']
            top_params[i-1][1] = results["params"][candidate]['theta0']

In [8]:
def predictAll(theta, nugget, trainX, trainY, testX, testY, testSet, title, _reg, _corr):
    if _corr == 'squared_exponential' or _corr == 'generalized_exponential':
        rbf_thetaUU = [ 12, 5, 5, 7, 46.1, 126, 544.2, 45, 101.25, 8, 9, 8, 984517, 1, 0.5263, 0.5, 2]
        rbf_thetaLL = [ 5, 1, 1, 1, 0.1, 113, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 2, 6456, 0.0001, 0.0001, 0.0001, 1]
        rbf_theta0 = [ 8.4196, 2.5759, 1.5982, 2.3259, 6.1810, 120.2545, 169.2713, 36.9830, 13.9196, 3.0348, 3.0268, 3.9196, 102582.2589, 0.4433, 0.1843, 0.1086, 1.3750]
        rbf_thetaU = np.add((np.subtract(rbf_thetaUU, rbf_theta0)), rbf_theta0)
        rbf_thetaL = np.subtract(rbf_theta0, (np.subtract(rbf_theta0, rbf_thetaLL)))
        print(rbf_thetaU)
        print(rbf_thetaL)
        rbf_nugget = [3.285953444, 2.02995456, 0.642139668, 1.95182956, 65.89152618, 9.153997927, 21360.96237, 705.4621752, 145.8506856, 4.786432338, 6.472496811, 1.886399872, 14672819859, 0.045545572, 0.017114311, 0.008621105, 0.234375]
        
        gp = gaussian_process.GaussianProcess(regr=_reg, corr=_corr, theta0=rbf_theta0, thetaL = rbf_thetaL, thetaU = rbf_thetaU)
    else:
        gp = gaussian_process.GaussianProcess(regr=_reg, corr=_corr, theta0=theta, nugget =nugget)
    gp.fit(trainX, trainY)
    
    predictedY, MSE = gp.predict(testX, eval_MSE = True)
    sigma = np.sqrt(MSE)
    
    results = {}
    results['predictedY'] = predictedY
    results['sigma'] = sigma

    trainScore = gp.score(trainX, trainY)
    testScore = sklearn.metrics.r2_score(testY, predictedY)
    print ("Train score R2:", trainScore)
    print ("Test score R2:", testScore)
    
    return trainScore, testScore, results

In [9]:
def calculateError(predict, actual) :
    total =0.0
    for i in range(len(predict)):
        total += predict[i] - actual[i]
    return total/len(predict)

In [10]:
def runGP (_reg, _corr):
    clf = gaussian_process.GaussianProcess(regr=_reg, corr=_corr)
    print(clf.get_params().keys())
    
    grid_search = GridSearchCV(clf, param_grid=param_grid)
    start = time()
    grid_search.fit(X_train_smooth, y_train)
    print("BEST PARAMS: ", grid_search.best_params_)
    report(grid_search.cv_results_, top_params)
    print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
          % (time() - start, len(grid_search.cv_results_['params'])))

    highestTest = 0;
    highestTest_Train = 0;
    highestTest_Error = 0;
    highestTest_theta = 0;
    highestTest_nugget = 0;
    
    for i in range(num_top):
        trainScore, testScore, results = predictAll(top_params[i][1], top_params[i][0], x_train, y_train, 
                                      x_test, y_test, x_test, 'Highest Speed Figure', _reg, _corr)
        err = calculateError(results['predictedY'],y_test)
        
        if testScore > highestTest:
            highestTest = testScore
            highestTest_Train = trainScore
            highestTest_Error = err
            highestTest_theta = top_params[i][1]
            highestTest_nugget = top_params[i][0]
          
    print()
    print("Highest Test Score: ", highestTest)
    print("Accompanying training Score: ", highestTest_Train)
    print("Accompanying error: ", highestTest_Error)
    print("Accompanying theta & nugget: ", highestTest_theta, " ", highestTest_nugget)
    print()

In [11]:
for j, reg in enumerate(reg_types):
    for k, corr in enumerate(corr_types):
        print ("Regression Model: ", reg, " Correlation Model: ", corr)
        runGP(reg,corr)

Regression Model:  constant  Correlation Model:  linear
dict_keys(['nugget', 'verbose', 'thetaL', 'storage_mode', 'random_start', 'optimizer', 'beta0', 'normalize', 'regr', 'corr', 'theta0', 'random_state', 'thetaU'])
BEST PARAMS:  {'theta0': 0.005, 'nugget': 0.02}
Model with rank: 1
Mean validation score: 0.776 (std: 0.037)
Parameters: {'theta0': 0.005, 'nugget': 0.02}

Model with rank: 2
Mean validation score: 0.776 (std: 0.036)
Parameters: {'theta0': 0.005, 'nugget': 0.015}

Model with rank: 3
Mean validation score: 0.775 (std: 0.038)
Parameters: {'theta0': 0.005, 'nugget': 0.025}

Model with rank: 4
Mean validation score: 0.773 (std: 0.039)
Parameters: {'theta0': 0.005, 'nugget': 0.03}

Model with rank: 5
Mean validation score: 0.773 (std: 0.032)
Parameters: {'theta0': 0.01, 'nugget': 0.03}

GridSearchCV took 2.09 seconds for 84 candidate parameter settings.
Train score R2: 0.95286062821
Test score R2: 0.80808810982
Train score R2: 0.960691107179
Test score R2: 0.818168460745
Train

In [12]:
trainScore, testScore, results = predictAll(0, 0, x_train, y_train, 
                                      x_test, y_test, x_test, 'Highest Speed Figure', 'linear', 'squared_exponential')
print(trainScore)
print(testScore)

[  1.20000000e+01   5.00000000e+00   5.00000000e+00   7.00000000e+00
   4.61000000e+01   1.26000000e+02   5.44200000e+02   4.50000000e+01
   1.01250000e+02   8.00000000e+00   9.00000000e+00   8.00000000e+00
   9.84517000e+05   1.00000000e+00   5.26300000e-01   5.00000000e-01
   2.00000000e+00]
[  5.00000000e+00   1.00000000e+00   1.00000000e+00   1.00000000e+00
   1.00000000e-01   1.13000000e+02   1.00000000e-04   1.00000000e-04
   1.00000000e-04   1.00000000e-04   1.00000000e-04   2.00000000e+00
   6.45600000e+03   1.00000000e-04   1.00000000e-04   1.00000000e-04
   1.00000000e+00]
Train score R2: 1.0
Test score R2: 0.104972626719
1.0
0.104972626719
